In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# Plotly libraries
import plotly
import plotly.express as px
import plotly.graph_objs as go
import chart_studio.plotly as py

import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode, plot
cf.go_offline()

import warnings
warnings.filterwarnings('ignore')

In [2]:
cc_train = pd.read_csv('Edureka_Consumer_Complaints_train.csv')
cc_test = pd.read_csv('Edureka_Consumer_Complaints_test.csv')

In [3]:
cc_train.head(2)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2015-10-14,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,NaN,Equifax,GA,30134,NaN,Consent not provided,Web,2015-10-14,Closed with explanation,Yes,No,1605653
1,2015-04-26,Bank account or service,Other bank product/service,Deposits and withdrawals,NaN,RE : XXXX XXXX XXXX-PRIVILEGED AND CONFIDENTIA...,NaN,Wells Fargo & Company,GA,319XX,NaN,Consent provided,Web,2015-04-26,Closed with explanation,Yes,Yes,1347613


In [4]:
#Analyze the missing values in the columns
cc_train.isnull().sum()

Date received                        0
Product                              0
Sub-product                     103786
Issue                                0
Sub-issue                       219374
Consumer complaint narrative    302630
Company public response         290879
Company                              0
State                             2903
ZIP code                          2911
Tags                            308584
Consumer consent provided?      257230
Submitted via                        0
Date sent to company                 0
Company response to consumer         0
Timely response?                     0
Consumer disputed?                   0
Complaint ID                         0
dtype: int64

In [5]:
#Analyze the missing values in the columns
cc_train.columns =cc_train.columns.str.title()
mode_value= cc_train['Consumer Disputed?'].mode()
mode_value ='No'
cc_train['Consumer Disputed?'].fillna(mode_value, inplace=True)
cc_train['Consumer Disputed?'].isnull().fillna(mode_value,inplace =True)
cc_train.isnull().mean().round(4)*100

Date Received                    0.00
Product                          0.00
Sub-Product                     28.93
Issue                            0.00
Sub-Issue                       61.14
Consumer Complaint Narrative    84.34
Company Public Response         81.07
Company                          0.00
State                            0.81
Zip Code                         0.81
Tags                            86.00
Consumer Consent Provided?      71.69
Submitted Via                    0.00
Date Sent To Company             0.00
Company Response To Consumer     0.00
Timely Response?                 0.00
Consumer Disputed?               0.00
Complaint Id                     0.00
dtype: float64

In [6]:
# getting the sum of null values and ordering.
total = cc_train.isnull().sum().sort_values(ascending = False)  

#getting the percent and order of null.
percent = (cc_train.isnull().sum()/cc_train.isnull().count()*100).sort_values(ascending =False)

# Concatenating the total and percent
df = pd.concat([total , percent],axis =1,keys=['Total' ,'Percent'])

# Returning values of nulls different of 0
(df[~(df['Total'] == 0)])

,Total,Percent
Tags,308584,86.002062
Consumer Complaint Narrative,302630,84.342688
Company Public Response,290879,81.067696
Consumer Consent Provided?,257230,71.689752
Sub-Issue,219374,61.139322
Sub-Product,103786,28.925058
Zip Code,2911,0.811293
State,2903,0.809063


In [7]:
cc_train[['Issue','Date Received','Product','Sub-Issue','Consumer Complaint Narrative','Company',
               'Company Public Response','Consumer Consent Provided?',
               'Company Response To Consumer','Submitted Via']].describe().transpose()

,count,unique,top,freq
Issue,358810,95,"Loan modification,collection,foreclosure",60185
Date Received,358810,1758,2015-08-27,579
Product,358810,12,Mortgage,117066
Sub-Issue,139436,67,Account status,17142
Consumer Complaint Narrative,56180,55535,This company continues to report on my credit ...,26
Company,358810,3064,Bank of America,34903
Company Public Response,67931,10,Company chooses not to provide a public response,31526
Consumer Consent Provided?,101580,4,Consent provided,56181
Company Response To Consumer,358810,6,Closed with explanation,265700
Submitted Via,358810,6,Web,235462


In [8]:
grouped = cc_train.groupby(['Company Response To Consumer']).size()
pie_chart = go.Pie(labels=grouped.index,values=grouped,
                  title='Company Response to the Customer')
iplot([pie_chart])

In [9]:
states = cc_train['State'].value_counts()

scl = [
    [0.0, 'rgb(242,240,247)'],
    [0.2, 'rgb(218,218,235)'],
    [0.4, 'rgb(188,189,220)'],
    [0.6, 'rgb(158,154,200)'],
    [0.8, 'rgb(117,107,177)'],
    [1.0, 'rgb(84,39,143)']
]

data = [go.Choropleth(
    colorscale = scl,
    autocolorscale = False,
    locations = states.index,
    z = states.values,
    locationmode = 'USA-states',
    text = states.index,
    marker = go.choropleth.Marker(
        line = go.choropleth.marker.Line(
            color = 'rgb(254,254,254)',
            width = 2
        )),
    colorbar = go.choropleth.ColorBar(
        title = "Complaints")
)]

layout = go.Layout(
    title = go.layout.Title(
        text = 'Complaints by State'
    ),
    geo = go.layout.Geo(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = True,
        lakecolor = 'rgb(100,149,237)'),
)

fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [10]:
pd.crosstab(cc_train['Timely Response?'],cc_train['Submitted Via']).iplot(kind='bar',
                                                                                    title='Company Response to the Customer')

In [11]:
pd.crosstab(cc_train['Timely Response?'], cc_train['Consumer Disputed?']).iplot(kind='bar',
                                                                    title ='Timely Response vs Consumer Disputed' )

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder = LabelEncoder()
onehotencoder =OneHotEncoder()

In [13]:
cc_train['Consumer_encode']= labelencoder.fit_transform(cc_train['Consumer Disputed?'])

In [14]:
enc = OneHotEncoder(handle_unknown='ignore')
consumer_data1 = pd.DataFrame(enc.fit_transform(cc_train[['Product']]).toarray())
df =cc_train.join(consumer_data1)

In [ ]:
x = df.iloc[:,24:41].values
y = df['Consumer_encode'].values

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y, test_size =0.25, random_state =10)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train =sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=10,criterion='entropy', random_state =10)
classifier.fit(x_train,y_train)

In [ ]:
y_pred = classifier.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
print(cm)

In [ ]:
from sklearn.metrics import accuracy_score
print('Accuracy Score:',accuracy_score(y_test,y_pred))